# Data Preparation

`Overview`
This notebook handles the initial data processing pipeline:
- Loading raw data from source files
- Performing exploratory data analysis (EDA)
- Cleaning and handling missing values
- Feature preprocessing and engineering
- Exporting processed datasets for modeling

`Inputs`
- Raw data files from `../data/raw/` 

`Outputs`
- Processed datasets in `../data/processed/`
- EDA visualizations in `../reports/figures/`

`Dependencies`
- pandas
- numpy
- matplotlib
- seaborn

*Note: This is notebook 1 of the analysis pipeline*

In [14]:
# Imports 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime
from pprint import pprint
from pathlib import Path
import csv
import sys

# Import data classes
project_root = Path.cwd().parent  # assumes you're in /notebooks
sys.path.append(str(project_root))

# Import custom modules
from backend.etl.ingestion import inspect_bad_lines, auto_fix_row, robust_csv_reader
from backend.etl.cleaning import standardize_columns, clean_numeric_column, clean_date_column


In [6]:
!which python


'which' is not recognized as an internal or external command,
operable program or batch file.


Here we load the project specific datasets as CSV files. In the follow-up cell, we load the auxiliary dataset containing extra information on the CORDIS-HORIZON projects. This includes
- Scientific vocabulary 
- legal basis documents
- organization
- project
- topics
- webItem 
- webLink

In [3]:
run_dir = os.getcwd()
print(run_dir)

/Users/bertdepoorter/Nextcloud/EU_Horizon_Dashboard/notebooks


In [7]:
# Import the dataset as pandas DataFrame
run_dir = os.getcwd()
parent_dir = os.path.dirname(run_dir)

raw_dir = f'{parent_dir}/data/raw'
interim_dir = f'{parent_dir}/data/interim'
processed_dir = f'{parent_dir}/data/processed'

# define file paths to project-specific files
data_report_path = f'{raw_dir}/reportSummaries.csv'
data_filereport_path = f'{raw_dir}/file_report.csv'
data_publications_path = f'{raw_dir}/projectPublications.csv'
data_deliverables_path = f'{raw_dir}/projectDeliverables.csv'



## Define functions for cleaning
The following functions are necessary to load the datasets correctly without manually changing them.
- `inspect_bad_lines`: inspect lines that cannot be read directly
- `auto_fix_row`: function that fixes row by merging excess columns together
- `robust_csv_reader`: robust function that loads CSV files while applying `auto_fix_row` function on the bad lines

Usage:
```
# check bad lines
project_df, problematic_lines = inspect_bad_lines(project_path)

# INspect how many bad lines there are 
print(f"DataFrame loaded with {len(project_df)} rows.")
print(f"Number of problematic lines: {len(problematic_lines)}")
```

## Inspect Reports

In [8]:
# get DataFrame keys
data_report = pd.read_csv(data_report_path, delimiter=';')


### Missing values



In [9]:
# look for missing values
report_missing = data_report.isnull()

# check which columns are missing data
for key in data_report:
    missing = report_missing[report_missing[key] == True]
    print(f'For key {key}:\n     {len(missing.id)} elements are missing.')

For key id:
     0 elements are missing.
For key title:
     0 elements are missing.
For key projectID:
     0 elements are missing.
For key projectAcronym:
     0 elements are missing.
For key attachment:
     1861 elements are missing.
For key contentUpdateDate:
     0 elements are missing.
For key rcn:
     0 elements are missing.


## Inspect deliverables

In [10]:
# Inspect Dataframe
# account for problematic lines

deliverables_df, problematic_lines = inspect_bad_lines(data_deliverables_path)

print(f"DataFrame loaded with {len(deliverables_df)} rows.")
print(f"Number of problematic lines: {len(problematic_lines)}")
    

Found 11 problematic lines. Displaying the first 5:
Line 1416: ['101081964_11_DELIVHORIZON', 'Guideline for trial implementation', 'Documents, reports', 'This guideline will summarise and present agricultural practices performed at BioMonitor4CAP farms and research sites with links to local/regional nature conservation goals and targeted species. Based on these baseline information the guideline will present how, when and to what extend research groups of WPs 2 and 4 are integrated into the activities of WP3. Supports specifically achieving following outcomes (Part B, section 2.3): A and B ""Strategy on monitoring soil biodiversity at farm scale adopted by science, end users, and policy"" and E ""Roadmap on expanding enhancing application and implementation of agri-environmental measures committing to the preservation of biodiversity', ' particularly agroforestry."" This deliverable is an output of task 3.1.""', '101081964', 'BioMonitor4CAP', 'https://ec.europa.eu/research/participants

In [11]:
# Try loaoding with the robust CSV rreader
data_deliverables = robust_csv_reader(data_deliverables_path, expected_columns=10, problematic_column=3)


### Missing values
Here we handle the missing values in the dataset

We are missing elements in the following columns:
- deliverableType
    - option 1: change to `'other'`
    - option 2: look up individual titles and add manually
- description
    - option 1: add empty string
    - Inspect manually to gain more insight what they exactly represent
        - Update: all the titles related to the projects are quite related. I suggest we copy title values into the description column.
- url
    - 1 missing url. Add the url to the main page of this project (SELFY, id = 101069748_16_DELIVHORIZON) instead of link to deliverable?
- rcn
    - 1 rcn is missing. 
    - Looked this number up in publication list based on the projectAcronym = `'GeneBEcon'`. There the rcn number is gives as `1077637.0`


In [ ]:
# clean the dataframe


## Inspect Publications

In [71]:
# Inspect Dataframe
data_publications = pd.read_csv(data_publications_path, delimiter=';')


In [22]:
publications_df, problematic_lines = inspect_bad_lines(data_publications_path, expected_columns=16)

print(f"DataFrame loaded with {len(publications_df)} rows.")
print(f"Number of problematic lines: {len(problematic_lines)}")

DataFrame loaded with 24150 rows.
Number of problematic lines: 0


### Missing values
Here we inspect the missing data in this file, and outline how we are goiing to treat these missing data points

In [24]:
# look for missing values
publications_missing = data_publications.isnull()

# check which columns are missing data
for key in publications_missing.keys():
    missing = publications_missing[publications_missing[key] == True]
    if len(missing.id) > 0:
        print(f'For key {key}:\n     {len(missing.id)} elements are missing.')

For key authors:
     75 elements are missing.
For key journalTitle:
     2099 elements are missing.
For key journalNumber:
     13622 elements are missing.
For key publishedPages:
     24142 elements are missing.
For key issn:
     7004 elements are missing.
For key isbn:
     23219 elements are missing.
For key doi:
     2293 elements are missing.


There is quite some missing data in this file. Let us go through each line individually.
- authors:
    - This sucks. Would have been very nice to decompose author strings into single authors and make the connections
    - How to treat this: look into the article title string to check whether this one contains more author infromation
- journalTitle:
    - chack in the publication title. Sometimes there one has just copy-pasted the whole article reference
- journalNumber:
    - Not the most relevant parameter in my opinion. Just make all NaN zeros
- publishedYear:
    - Manually look this up
- publishedPages:
    - Not the most relevant parameter in my opinion. Just make all NaN zeros
- issn:
    - Not the most relevant parameter in my opinion. Just make all NaN zeros
- isbn:
    - Not the most relevant parameter in my opinion. Just make all NaN zeros
- doi:
    - Fuck this, just pass about:blank as url. 
- rcn:
    - Manually adjust this one. 
        - Update: this entry was missing an entry for authors, all following field shifted 1 column to the left. Manually fixed this one. 



The cell above does give an empty DataFrame. Used it to get the information to look up rcn number in other datasets.

In [ ]:
# test cleaning function

In [39]:
# Save to intermediate
data_publications.to_csv(f'{interim_dir}/projectPublications_interim.csv', sep=';')

# Inspect CORDIS-HORIZON projects data files
This is the folder containing some more datasets on the different projects.

### Define data paths

In [5]:
# define file paths
SciVoc_path = f'{raw_dir}/euroSciVoc.csv'
legalBasis_path = f'{raw_dir}/legalBasis.csv'
organization_path = f'{raw_dir}/organization.csv'
project_path = f'{raw_dir}/project.csv'
topics_path = f'{raw_dir}/topics.csv'
webItems_path = f'{raw_dir}/webItem.csv'
webLink_path = f'{raw_dir}/webLink.csv'

In [6]:
# Import some informative files

# load datasets
read_csv_options = {
    "delimiter": ";",
    "quotechar": '"',
    "escapechar": "\\",
    'doublequote': False,
    # "on_bad_lines": "skip",   # we skip lines that do not import properly for now
    "engine": "python"  # 'python' engine handles complex parsing better
}


## SciVoc dataset

In [ ]:
# load
sci_voc_df = pd.read_csv(SciVoc_path, **read_csv_options)

# clean
from backend.etl.cleaning import clean_scivoc
sci_voc_cleaned = clean_scivoc(sci_voc_df)

# save to interim
sci_voc_cleaned.to_csv(f'{interim_dir}/euroSciVoc_interim.csv', sep=';')

## organization dataset

In [ ]:
# load
organization_df = pd.read_csv(organization_path, delimiter=';')

# clean
from backend.etl.cleaning import clean_organization
organization_cleaned = clean_organization(organization_df)

# save to interim
organization_cleaned.to_csv(f'{interim_dir}/organization_interim.csv', sep=';')

## topics dataset

In [ ]:
# load
topics_df = pd.read_csv(topics_path, **read_csv_options)

# clean
from backend.etl.cleaning import clean_topics
topics_cleaned = clean_topics(topics_df)

# save to interim
topics_cleaned.to_csv(f'{interim_dir}/topics_interim.csv', sep=';')

NameError: name 'topics_path' is not defined

## Legal Basis dataset

In [ ]:
# load
legal_basis_df = pd.read_csv(legalBasis_path, **read_csv_options)

# clean
from etl.cleaning import clean_legalbasis, standardize_columns
legal_basis_cleaned = clean_legalbasis(legal_basis_df)

# save to interim
legal_basis_cleaned.to_csv(f'{interim_dir}/legalBasis_interim.csv', sep=';')

/Users/bertdepoorter/Nextcloud/EU_Horizon_Dashboard/backend/etl/cleaning.py:192: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['uniqueprogrammepart'] = df['uniqueprogrammepart'].fillna(False)


## webItem / webLink dataset

In [ ]:
# load
web_items_df = pd.read_csv(webItems_path, **read_csv_options)
web_link_df = pd.read_csv(webLink_path, **read_csv_options)

# clean

from etl.cleaning import clean_webitem, clean_weblink
web_items_cleaned = clean_webitem(web_items_df)
web_link_cleaned = clean_weblink(web_link_df)

# save to interim
web_items_cleaned.to_csv(f'{interim_dir}/webItems_interim.csv', sep=';')
web_link_cleaned.to_csv(f'{interim_dir}/webLink_interim.csv', sep=';')

## projects dataset

In [14]:
project_df, problematic_lines = inspect_bad_lines(project_path, expected_columns=20)

print(f"DataFrame loaded with {len(project_df)} rows.")
print(f"Number of problematic lines: {len(problematic_lines)}")

Found 127 problematic lines. Displaying the first 5:
Line 11: ['101114248', 'ELIA', 'CLOSED', 'Elia - Smart Assistant for English Learning', '2023-07-01', '2024-06-30', '0', '75000', 'HORIZON.3.2', 'HORIZON-EIE-2022-SCALEUP-02-02', '2023-06-05', 'HORIZON', 'HORIZON-EIE-2022-SCALEUP-02', 'HORIZON-EIE-2022-SCALEUP-02', 'HORIZON-CSA', '', "We are a startup founded exclusively by women holding the top positions CEO and CTO. Our vision is to reduce inequalities by helping people own their English. That's why we created a personal assistant – Elia. Elia is a tool for busy professionals or swamped students struggling with their English. It connects English learning to their daily activities, e.g. writing an email at work", ' watching videos on YouTube', ' or reading an article for a biology class. Because learning that is personalised and in context has been found to be the most effective form of learning. Elia started as a PhD project. Hence it\'s based on insights from cognitive linguistics

In [15]:
project_df = robust_csv_reader(project_path, expected_columns=20, problematic_column=14)

In [ ]:
# clean the dataset
from backend.etl.cleaning import clean_project
project_cleaned = clean_project(project_df)

In [17]:
# save the dataframe to interim folder
project_df.to_csv(f'{interim_dir}/project_interim.csv', sep=';')

## Construct functions to access cleaned data

We now define some functions that allow easy access to all aspects of different projects. 


- Merge datasets into one object
- Standardize column names => they are compatible
- Create function that allow access to project-specific data:
    - function argument: project name / acronym / identifier
    - function output: data class with project information as attributes
    - Or: approach this from a class init perspective

Find some way to pass load datasets
apply class on this, without having to load the full dataset each time we initialize the class


In [15]:
import sys
import os
from pathlib import Path

# Add the project root directory to sys.path so absolute imports work


from backend.classes import CORDIS_data

parent_dir = project_root
Data_structure = CORDIS_data(parent_dir)

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\Suleyman Ismayilov\\Desktop\\Suleyman\\EU_Horizon_Dashboard/data/interim/projectdeliverables_interim.csv'

In [ ]:
# store feature-enriched dataframe to the processed directory
Data_structure.export_dataframes(f'{processed_dir}/')

In [72]:
Data_structure.list_of_acronyms()

,0
0,PvSeroRDT
1,BIOBoost
2,GlycanTrigger
3,CHIKVAX_CHIM
4,The Oater
...,...
15048,EUCYS2022
15049,RESAVER_2023
15050,Leiden2022-ECS-ESOF
15051,EUCYS2024


In [95]:
class Project_data(CORDIS_data):
    def __init__(self, project_id=None, acronym=None):
        # Inherit from CORDIS_data by initialzing parent class
        super().__init__()

        # Check if both project_id and acronym are provided
        self.id, self.acronym = self._resolve_project_id_acronym(project_id, acronym)

        # Add all project-specific data as attributes
        
        self.project_info = self._get_project_info()
        self.publications = self._get_publications()
        self.deliverables = self._get_deliverables()
        self.organizations = self._get_organizations()
        self.scivoc = self._get_scivoc()
        self.topics = self._get_topics()
        self.legal_basis = self._get_legal_basis()

        # Add some enriched data
        self.temporal_features = self._compute_temporal_features()
        self.people_institutions = self._compute_people_institutions()
        self.financial_metrics = self._compute_financial_metrics()
        self.scientific_thematic = self._compute_scientific_thematic()

    def _resolve_project_id_acronym(self, project_id, acronym):
        if project_id is not None and acronym is not None:
            expected_acronym = self.project_df[self.project_df['id'] == project_id]['acronym'].values[0]
            if expected_acronym != acronym:
                raise ValueError(f"Acronym mismatch: ID {project_id} is linked to {expected_acronym}, not {acronym}.")
        elif acronym is not None:
            project_id = self.project_df[self.project_df['acronym'] == acronym]['id'].values[0]
        elif project_id is not None:
            acronym = self.project_df[self.project_df['id'] == project_id]['acronym'].values[0]
        else:
            raise ValueError("Provide at least one of project_id or acronym.")
        return project_id, acronym

    def _get_project_info(self):
        return self.project_df[self.project_df['id'] == self.id].iloc[0].to_dict()

    def _get_publications(self):
        return self.data_publications[self.data_publications['projectID'] == self.id]

    def _get_deliverables(self):
        return self.data_deliverables[self.data_deliverables['projectID'] == self.id]

    def _get_organizations(self):
        return self.organization_df[self.organization_df['projectID'] == self.id]

    def _get_scivoc(self):
        return self.sci_voc_df[self.sci_voc_df['projectID'] == self.id]

    def _get_topics(self):
        return self.topics_df[self.topics_df['projectID'] == self.id]

    def _get_legal_basis(self):
        return self.legal_basis_df[self.legal_basis_df['projectID'] == self.id]
    

    # Add additional project features
    def _compute_temporal_features(self):
        fmt = "%Y-%m-%d"
        start = self.project_info.get("startDate", None)
        end = self.project_info.get("endDate", None)
        ec_sig = self.project_info.get("ecSignatureDate", None)

        try:
            start_date = datetime.strptime(start, fmt)
            end_date = datetime.strptime(end, fmt)
            duration_days = (end_date - start_date).days
        except:
            duration_days = None

        return {
            "start_year": start.year if start else None,
            "end_year": end.year if end else None,
            "signature_year": ec_sig[:4] if ec_sig else None,
            "duration_days": duration_days
        }
    
    def _compute_people_institutions(self):
        orgs = self.organizations
        if orgs.empty:
            return {}
        country_counts = orgs["country"].value_counts().to_dict()
        activity_types = orgs["activityType"].value_counts().to_dict()
        n_partners = orgs["organisationID"].nunique()

        return {
            "n_partners": n_partners,
            "countries": country_counts,
            "activity_types": activity_types
        }

    def _compute_financial_metrics(self):
        ec_total = self.project_info.get("ecMaxContribution", None)
        total_cost = self.project_info.get("totalCost", None)
        ec_partner_sum = self.organizations["ecContribution"].sum()
        cost_partner_sum = self.organizations["totalCost"].sum()

        try:
            ec_per_deliverable = ec_total / len(self.deliverables)
        except:
            ec_per_deliverable = None

        try:
            ec_per_publication = ec_total / len(self.publications)
        except:
            ec_per_publication = None

        return {
            "ec_total": ec_total,
            "total_cost": total_cost,
            "ec_sum_from_partners": ec_partner_sum,
            "cost_sum_from_partners": cost_partner_sum,
            "ec_per_deliverable": ec_per_deliverable,
            "ec_per_publication": ec_per_publication
        }

    def _compute_scientific_thematic(self):
        scivoc_titles = self.scivoc['euroSciVocTitle'].dropna().unique().tolist()
        topic_titles = self.topics['title'].dropna().unique().tolist()

        pub_types = self.publications['isPublishedAs'].value_counts().to_dict()
        deliverable_types = self.deliverables['deliverableType'].value_counts().to_dict()

        return {
            "scivoc_keywords": scivoc_titles,
            "topic_keywords": topic_titles,
            "publication_types": pub_types,
            "deliverable_types": deliverable_types
        }
    def summary(self):
        return {
            "project_id": self.id,
            "acronym": self.acronym,
            "title": self.project_info.get("title", ""),
            "temporal": self.temporal_features,
            "institutions": self.people_institutions,
            "financials": self.financial_metrics,
            "keywords": self.scientific_thematic
        }
    
    def inspect_project_data(self):
        """
        Print or return a structured overview of all enriched data for the selected project.
        """
        if not hasattr(self, 'id') or not hasattr(self, 'acronym'):
            raise AttributeError("Please set a project using the `project()` method first.")

        print(f"\nProject: {self.acronym} (ID: {self.id})")
        print("="*60)

        print("\nPublications:")
        pprint(getattr(self, 'publications', {}), indent=4)

        print("\nDeliverables:")
        pprint(getattr(self, 'deliverables', {})[['deliverableType', 'description']], indent=4)

        print("\nInstitutions / Organizations:")
        pprint(getattr(self, 'organizations', []), indent=4)

        print("\nFinancial Info:")
        pprint({
            'Total Cost': getattr(self, 'total_cost', None),
            'EC Contribution': getattr(self, 'ec_contribution', None),
            'Num Orgs': getattr(self, 'num_organizations', None),
            'Countries': getattr(self, 'countries', None),
        }, indent=4)

        print("\nDates & Duration:")
        pprint({
            'Start Date': getattr(self, 'start_date', None),
            'End Date': getattr(self, 'end_date', None),
            'Duration (days)': getattr(self, 'duration_days', None),
            'Year': getattr(self, 'year', None)
        }, indent=4)

        print("\nLegal & Administrative:")
        pprint({
            'Legal Basis': getattr(self, 'legal_basis', None),
            'Funding Scheme': getattr(self, 'funding_scheme', None),
            'Framework Programme': getattr(self, 'framework_programme', None),
        }, indent=4)

        print("\n🔬 Scientific Keywords (euroSciVoc):")
        pprint(getattr(self, 'sci_keywords', []), indent=4)

        print("\nProject Topics:")
        pprint(getattr(self, 'topics', []), indent=4)

        print("\nWeb Links:")
        pprint(getattr(self, 'web_links', []), indent=4)


In [90]:
p = Project_data(acronym="CLIMB")
summary = p.summary()
print(summary["financials"])

{'ec_total': 1622273.0, 'total_cost': 1622273.0, 'ec_sum_from_partners': np.float64(1622273.0), 'cost_sum_from_partners': '1622273', 'ec_per_deliverable': None, 'ec_per_publication': None}


Use pprint to get out the background information in a readable format.

In [91]:
pprint(summary["financials"])

{'cost_sum_from_partners': '1622273',
 'ec_per_deliverable': None,
 'ec_per_publication': None,
 'ec_sum_from_partners': np.float64(1622273.0),
 'ec_total': 1622273.0,
 'total_cost': 1622273.0}


In [96]:
# Inspect a certain project
p = Project_data(acronym="BIOBoost")
p.inspect_project_data()


Project: BIOBoost (ID: 101096150)

Publications:
Empty DataFrame
Columns: [Unnamed: 0, id, title, isPublishedAs, authors, journalTitle, journalNumber, publishedYear, publishedPages, issn, isbn, doi, projectID, projectAcronym, collection, contentUpdateDate, rcn]
Index: []

Deliverables:
                              deliverableType  \
18191                      Documents, reports   
18192                      Documents, reports   
18193  Websites, patent fillings, videos etc.   
18194                      Documents, reports   
18195                    Data Management Plan   
18196                      Documents, reports   

                                             description  
18191  The project management handbook will provide c...  
18192  The PDEC provides information to all project p...  
18193  Interactive online tool showing and mapping ma...  
18194  The report includes information on innovation ...  
18195  The DMP provides clear information on the cons...  
18196  Assessm